In [ ]:
import json
import uuid
import os
import json
from dotenv import load_dotenv
from pathlib import Path
from kafka import KafkaProducer
from faker import Faker
from time import sleep

In [ ]:
dotenv_path = Path('/resources/.env')
load_dotenv(dotenv_path=dotenv_path)

In [ ]:
kafka_host = os.getenv('KAFKA_HOST')
topic = os.getenv('KAFKA_TOPIC_NAME')
num_replication = int(os.getenv('KAFKA_REPLICATION'))
num_partitions = int(os.getenv('KAFKA_PARTITION'))
schema_registry_host = os.getenv('SCHEMA_REG_HOST')

In [ ]:
sys.path.append('./protobuf')
import protobuf_schema_pb2 

In [ ]:
bootstrap_servers = f'{kafka_host}:9092' 
schema_registry_url = f'http://{schema_registry_host}:8081' 
schema_registry_client = SchemaRegistryClient({'url': schema_registry_url})

In [ ]:
# Consuming and processing messages
for message in consumer:
    pet_data = message.value
    print(f"Consumed message: {pet_data}")
    
    # Perform calculation: Total cost of owning the pet
    total_cost = pet_data['price'] + (pet_data['weight'] * 10)
    
    # Print the calculated result
    print(f"Total cost of owning {pet_data['pet_name']} (Species: {pet_data['species']}, Breed: {pet_data['breed']}): ${total_cost}")